In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML

In [ ]:
pd_raw=pd.read_csv('https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_doses_admin_global.csv')

In [ ]:
pd_raw

In [ ]:
pd_raw.columns[:]

In [ ]:
time_idx=pd_raw.columns[12:]

In [ ]:
df_plot= pd.DataFrame({ 'Date':
                       time_idx
    
})
df_plot.head()

In [ ]:
pd_raw['Country_Region'].unique()

In [ ]:
pd_raw['Country_Region']

In [ ]:
pd_raw[pd_raw['Country_Region']=='US']

In [ ]:
import numpy as np
country_list=['Italy', 'US', 'Spain','Germany', 'Korea,South']

In [ ]:
for each in country_list:
    df_plot[each]=np.array(pd_raw[pd_raw['Country_Region']==each].iloc[:,12::].sum(axis=0))

In [ ]:
df_plot

In [ ]:
df_plot.set_index('Date').plot()

In [ ]:
# section 4.2 done Conversion of Date Objects

In [ ]:
df_plot.head()

In [ ]:
from datetime import datetime

In [ ]:
df_plot.Date[0]

In [ ]:
df_plot.to_csv('processed_vaccine.csv', sep=';', index=False)

In [ ]:
df_plot

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime 
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
mpl.rcParams['figure.figsize']=(16,9)
pd.set_option('display.max_rows', 500)
sns.set(style="darkgrid")

In [ ]:
df_plot = pd.read_csv("processed_vaccine.csv", sep = ';')
df_plot.sort_values('Date', ascending=False).head()

In [ ]:
fig = df_plot.iloc[15:,:].set_index('Date').plot()
fig.set_ylim(1,300000)
fig.set_yscale('log')   #Log scale is used for better visualization of exponential graph
fig.set_ylabel('vaccinated people (source: John hopkins csse) (log scale)')

In [ ]:
import plotly.graph_objects as go
import plotly

In [ ]:
country_list = ['US', 'Italy', 'Spain', 'Germany', 'Korea,South']

In [ ]:
fig = go.Figure()

# Define how to plot individual traces
for i in country_list:
    fig.add_trace(go.Scatter(x = df_plot['Date'], 
                             y = df_plot[i],
                             name= i,
                             mode='markers+lines',
                             line_width = 1,
                             marker_size = 3
                            )
                 )

# Defines the overall graph properties    
fig.update_layout(xaxis_title='Time', 
                  yaxis_title='Vaccinated people (source: John hopkins csse) (log scale)',
                  width = 1000, 
                  height = 800)
fig.update_yaxes(type = 'log')

fig.update_layout(xaxis_rangeslider_visible=True)

In [ ]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()

# Defines layout of the dashboard
app.layout = html.Div([
    
    html.Label('Multi-Select Country'),
   
    dcc.Dropdown(
        id='country_drop_down',
        options=[
            {'label': 'Italy', 'value': 'Italy'},
            {'label': 'US', 'value': 'US'},
            {'label': 'Spain', 'value': 'Spain'},
            {'label': 'Germany', 'value': 'Germany'},
            {'label': 'Korea,South', 'value': 'Korea,South'}
        ],
        value=['US', 'Germany'], # which are pre-selected
        multi=True
    ),   
        
    dcc.Graph(figure=fig, id='main_window_slope')
])

In [ ]:
from dash.dependencies import Input, Output

#Make dashboard interactive
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):
    traces = [] 
    for each in country_list:
        traces.append(dict(x=df_plot.Date,
                                y=df_plot[each],
                                mode='markers+lines',
                                opacity=0.8,
                                line_width=1,
                                marker_size=3, 
                                name=each
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                yaxis_title="Vaccinated people (source johns hopkins csse, log-scale)",
                xaxis={'title':'Timeline',
                        'tickangle':-30,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        
                      },
                yaxis={'title':'Vaccinated people (source johns hopkins csse, log-scale)',
                       'type':"log",
                       'range':'[1.1,5.5]'
                      }
        )
    }

In [ ]:
app.run_server(debug=True, use_reloader=False)